#### Snippet for making csv for diff things with different xvals

In [ ]:
### Passive RNN plot ###
normalize=True
ninfo = 1000
L_list = [128,256,512]

lambda_val = 0

normalize=True
plot_backwards=False


i = lambda_val
fig_num = 6
fig_let = string.ascii_lowercase[i]

xval_name = 'effective_temp'
csv_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', 'Lvar', f'fig{fig_num}{fig_let}.csv')
os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)

temp_list_fwd=[]
temp_list_bck=[]    
nn_list_fwd=[]
nn_list_bck=[]


for j, LX in enumerate(L_list):
    if LX == 256:

        folder_name = 'diff_lambda_merge' 
        mode = 'wetmerge'
        act_fwd_replace_list = [[], [0.004]]
        act_bck_replace_list = [[], [0.004]]
    else:
        folder_name = 'diff_lambda_periodic'
        mode = 'wet'
        act_fwd_replace_list = [[],[]] # [[], [0.004]]
        act_bck_replace_list = [[],[]] #[[], [0.004]]
        
    act_fwd_min_list = [0.04,0.006]
    act_bck_min_list = [0.04,0.006]
    act_fwd_max_list = [0.082,0.012]
    act_bck_max_list = [0.082,0.012]
    act_fwd_exl_list = [[], []] #[0.005]]
    act_bck_exl_list = [[], []]

    normalizer = LX if normalize else 1
    out_path_all = os.path.join(data_dir, f'{folder_name}\\{mode}\\na{LX}')
    out_path_all_wet = os.path.join(data_dir, f'diff_lambda_periodic\\wet\\na{LX}') 
    out_path_main = os.path.join(out_path_all, f'{lambda_val}')
    out_path_fwd = os.path.join(out_path_main, 'fwd')
    out_path_bck = os.path.join(out_path_main, 'bck')

    act_bck = np.load(os.path.join(out_path_bck, 'act_bck.npy'))
    act_fwd = np.load(os.path.join(out_path_fwd, 'act_fwd.npy'))
    def_arr_bck = np.load(os.path.join(out_path_bck, 'def_arr.npy'))
    def_arr_fwd = np.load(os.path.join(out_path_fwd, 'def_arr.npy'))
    nn_arr_av_fwd = np.load(os.path.join(out_path_fwd, 'nn_av.npy'))
    nn_arr_av_bck = np.load(os.path.join(out_path_bck, 'nn_av.npy'))

    out_path_main_wet = os.path.join(out_path_all_wet, f'{lambda_val}')
    out_path_fwd_wet = os.path.join(out_path_main_wet, 'fwd')
    out_path_bck_wet = os.path.join(out_path_main_wet, 'bck')
    act_bck_wet = np.load(os.path.join(out_path_bck_wet, 'act_bck.npy'))
    act_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'act_fwd.npy'))
    nn_arr_av_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'nn_av.npy'))
    nn_arr_av_bck_wet = np.load(os.path.join(out_path_bck_wet, 'nn_av.npy'))

    # apply replacements on nn_arr_av
    for act_replace in act_fwd_replace_list[i]:
        if act_replace in act_fwd:
            act_idx = np.argwhere(act_fwd == act_replace)[0][0]
            act_idx_wet = np.argwhere(act_fwd_wet == act_replace)[0][0]
        #     print(act_fwd[act_idx], act_fwd_wet[act_idx_wet])
            nn_arr_av_fwd[act_idx,:] = nn_arr_av_fwd_wet[act_idx_wet,:]
    for act_replace in act_bck_replace_list[i]:
        if act_replace in act_bck:
            act_idx = np.argwhere(act_bck == act_replace)[0][0]
            act_idx_wet = np.argwhere(act_bck_wet == act_replace)[0][0]
        #    print(act_bck[act_idx], act_bck_wet[act_idx_wet])
            nn_arr_av_bck[act_idx,:] = nn_arr_av_bck_wet[act_idx_wet,:]

    # make activity masks 
    act_fwd_mask = ((act_fwd >=  act_fwd_min_list[i]) & (act_fwd <=  act_fwd_max_list[i]))
    act_bck_mask = ((act_bck >=  act_bck_min_list[i]) & (act_bck <=  act_bck_max_list[i]))
    for act_excl in act_fwd_exl_list[i]:
        if act_excl in act_fwd:
            act_idx = np.argwhere(act_fwd == act_excl)[0][0]
            act_fwd_mask[act_idx] = False
    for act_excl in act_bck_exl_list[i]:
        if act_excl in act_bck:
            act_idx = np.argwhere(act_bck == act_excl)[0][0]
            act_bck_mask[act_idx] = False

    temp_list_fwd.append(act_fwd[act_fwd_mask])
    nn_list_fwd.append(nn_arr_av_fwd[act_fwd_mask,:])
    if plot_backwards:
        temp_list_bck.append(act_bck[act_bck_mask])
        nn_list_bck.append(nn_arr_av_bck[act_bck_mask,:])
        temp_list_bck_tot = np.unique(np.concatenate(temp_list_bck))    

temp_list_fwd_tot = np.unique(np.concatenate(temp_list_fwd))
xval_name= 'effective_temp'

df = pd.DataFrame(index=temp_list_fwd_tot)
# name index
df.index.name = xval_name


for j, LX in enumerate(L_list):
    temp_mask_fwd_ext = False * np.ones_like(temp_list_fwd_tot, dtype=bool)
    # if temp in temp_list_fwd[j], set mask to True
    for k, temp in enumerate(temp_list_fwd_tot):
        if temp in temp_list_fwd[j]:
            temp_mask_fwd_ext[k] = True
    col_name_av = f'rnn_av_fwd_(L={LX})'
    col_name_std = f'rnn_sem_fwd_(L={LX})'
    #print(temp_mask_fwd_ext)
    #print(nn_list_fwd[j, :,0], type(nn_list_fwd[j, :,0]))
    df[col_name_av] = np.nan
    df[col_name_std] = np.nan
    df.loc[temp_mask_fwd_ext, col_name_av] = nn_list_fwd[j][:,0]
   # print(nn_list_fwd[j][:,0], type(nn_list_fwd[j][:,0]))
    df.loc[temp_mask_fwd_ext, col_name_std] = nn_list_fwd[j][:,1]
   #

# set dtype to float 
df = df.astype(float)
df.to_csv(csv_save_path)
print('Saved csv to', csv_save_path)
df.head(15)

In [ ]:
### Passive RNN plot ###
normalize=True
ninfo = 1000
L_list = [256]

lambda_val = 1
normalize=True
plot_backwards=True


i = lambda_val
fig_num = 3
fig_let = 'e' if i==0 else 'f' #string.ascii_lowercase[i]

xval_name = 'effective_temp'
csv_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', f'{mode}', f'fig{fig_num}{fig_let}.csv')
os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)

out_path_main = os.path.join(out_path_all, f'{lambda_val}')
out_path_fwd = os.path.join(out_path_main, 'fwd')
out_path_bck = os.path.join(out_path_main, 'bck')

act_fwd_replace_list = [[], [0.004]]
act_bck_replace_list = [[], [0.004]]
act_fwd_min_list = [0.04,0.004]
act_bck_min_list = [0.04,0.004]
act_fwd_exl_list = [[], []] #[0.005]]
act_bck_exl_list = [[], []]

act_bck = np.load(os.path.join(out_path_bck, 'act_bck.npy'))
act_fwd = np.load(os.path.join(out_path_fwd, 'act_fwd.npy'))
lmax_av_fwd = np.load(os.path.join(out_path_fwd, 'lmax_av.npy')) / normalizer
lperc_av_fwd = np.load(os.path.join(out_path_fwd, 'lperc_av.npy')) / normalizer
lmax_av_bck = np.load(os.path.join(out_path_bck, 'lmax_av.npy')) / normalizer
lperc_av_bck = np.load(os.path.join(out_path_bck, 'lperc_av.npy')) / normalizer
nn_arr_fwd = np.load(os.path.join(out_path_fwd, 'nn_arr.npy'))
nn_arr_bck = np.load(os.path.join(out_path_bck, 'nn_arr.npy'))
def_arr_bck = np.load(os.path.join(out_path_bck, 'def_arr.npy'))
def_arr_fwd = np.load(os.path.join(out_path_fwd, 'def_arr.npy'))

out_path_main_wet = os.path.join(out_path_all_wet, f'{lambda_val}')
out_path_fwd_wet = os.path.join(out_path_main_wet, 'fwd')
out_path_bck_wet = os.path.join(out_path_main_wet, 'bck')
act_bck_wet = np.load(os.path.join(out_path_bck_wet, 'act_bck.npy'))
act_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'act_fwd.npy'))
lperc_av_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'lperc_av.npy')) / normalizer
lmax_av_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'lmax_av.npy')) / normalizer
lperc_av_bck_wet = np.load(os.path.join(out_path_bck_wet, 'lperc_av.npy')) / normalizer
lmax_av_bck_wet = np.load(os.path.join(out_path_bck_wet, 'lmax_av.npy')) / normalizer

 # apply replacements on nn_arr_av
for act_replace in act_fwd_replace_list[i]:
    if act_replace in act_fwd:
        act_idx = np.argwhere(act_fwd == act_replace)[0][0]
        act_idx_wet = np.argwhere(act_fwd_wet == act_replace)[0][0]
        print(act_fwd[act_idx], act_fwd_wet[act_idx_wet])
        lperc_av_fwd[act_idx,:] = lperc_av_fwd_wet[act_idx_wet,:]
        lmax_av_fwd[act_idx,:] = lmax_av_fwd_wet[act_idx_wet,:]
for act_replace in act_bck_replace_list[i]:
    if act_replace in act_bck:
        act_idx = np.argwhere(act_bck == act_replace)[0][0]
        act_idx_wet = np.argwhere(act_bck_wet == act_replace)[0][0]
        print(act_bck[act_idx], act_bck_wet[act_idx_wet])
        lperc_av_bck[act_idx,:] = lperc_av_bck_wet[act_idx_wet,:]
        lmax_av_bck[act_idx,:] = lmax_av_bck_wet[act_idx_wet,:]

# make activity masks 
act_fwd_mask = (act_fwd >=  act_fwd_min_list[i])
act_bck_mask = (act_bck >=  act_bck_min_list[i])
for act_excl in act_fwd_exl_list[i]:
    if act_excl in act_fwd:
        act_idx = np.argwhere(act_fwd == act_excl)[0][0]
        act_fwd_mask[act_idx] = False
for act_excl in act_bck_exl_list[i]:
    if act_excl in act_bck:
        act_idx = np.argwhere(act_bck == act_excl)[0][0]
        act_bck_mask[act_idx] = False

temp_list_fwd = act_fwd[act_fwd_mask]
temp_list_bck = act_bck[act_bck_mask]
#temp_list_fwd.append(act_fwd[act_fwd_mask])
lmax_fwd = lmax_av_fwd[act_fwd_mask,:]
lmax_bck = lmax_av_bck[act_bck_mask,:]
lperc_fwd = lperc_av_fwd[act_fwd_mask,:]
lperc_bck = lperc_av_bck[act_bck_mask,:]

print(lperc_fwd.shape, lperc_bck.shape)

#print(act_fwd[act_fwd_mask], act_bck[act_bck_mask])
temp_list_tot = np.unique(np.concatenate((act_fwd[act_fwd_mask], act_bck[act_bck_mask]))) if plot_backwards else np.unique(act_fwd[act_fwd_mask])
xval_name= 'effective_temp'

df = pd.DataFrame(index=temp_list_tot)
# name index
df.index.name = xval_name

for j, (direc, temp_list_dir, yval_list_dir) in enumerate(zip(['fwd', 'bck'], [temp_list_fwd, temp_list_bck], [lperc_fwd, lperc_bck ])):
    if direc == 'bck' and not plot_backwards:
        continue
    # extend temp_list_dir to total temp list
    temp_mask_ext = False * np.ones_like(temp_list_tot, dtype=bool)
    # if temp in temp_list_fwd[j], set mask to True
    for k, temp in enumerate(temp_list_tot):
        if temp in temp_list_dir:
            temp_mask_ext[k] = True
    col_name_av = f'lperc_av_{direc}'
    col_name_std = f'lperc_sem_{direc}'
  #  print(temp_list_tot, temp_list_dir, yval_list_dir.shape)
  #  print(temp_mask_ext.sum())
    df[col_name_av] = np.nan
    df[col_name_std] = np.nan
    df.loc[temp_mask_ext, col_name_av] = yval_list_dir[:,0]
    df.loc[temp_mask_ext, col_name_std] = yval_list_dir[:,1]

for j, (direc, temp_list_dir, yval_list_dir) in enumerate(zip(['fwd', 'bck'], [temp_list_fwd, temp_list_bck], [lmax_fwd, lmax_bck ])):
    if direc == 'bck' and not plot_backwards:
        continue
    # extend temp_list_dir to total temp list
    temp_mask_ext = False * np.ones_like(temp_list_tot, dtype=bool)
    # if temp in temp_list_fwd[j], set mask to True
    for k, temp in enumerate(temp_list_tot):
        if temp in temp_list_dir:
            temp_mask_ext[k] = True
    col_name_av = f'lmax_av_{direc}'
    col_name_std = f'lmax_sem_{direc}'
   # print(temp_list_tot, temp_list_dir, yval_list_dir.shape)
   # print(temp_mask_ext.sum())
    df[col_name_av] = np.nan
    df[col_name_std] = np.nan
    df.loc[temp_mask_ext, col_name_av] = yval_list_dir[:,0]
    df.loc[temp_mask_ext, col_name_std] = yval_list_dir[:,1]

# set dtype to float 
df = df.astype(float)
df.to_csv(csv_save_path)
print('Saved csv to', csv_save_path)
df.head(15)

#### Snippet for diff things, same xvals

In [ ]:
lambda_val = 1
normalize=True
ninfo = 1000

i = lambda_val
fig_num = 2
fig_let = 'j' #string.ascii_lowercase[i]

xval_name = 'radial_distance_normalized'
csv_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', f'{mode}', f'fig{fig_num}{fig_let}.csv')
os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)

normalizer = LX if normalize else 1
out_path_all = out_path_all = os.path.join(data_dir, f'{folder_name}\\{mode}\\na{LX}')

out_path_main = os.path.join(out_path_all, f'{lambda_val}')
out_path_fwd = os.path.join(out_path_main, 'fwd')
out_path_bck = os.path.join(out_path_main, 'bck')

act_include_list = [0.03, 0.045, 0.07] if lambda_val == 0 else [.003, 0.004, 0.01]
act_idx_fwd_include_list = [3, -6, -2,]# -1]

act_bck = np.load(os.path.join(out_path_bck, 'act_bck.npy'))
act_fwd = np.load(os.path.join(out_path_fwd, 'act_fwd.npy'))
rdf_arr_av_fwd = np.load(os.path.join(out_path_fwd, 'rdf_arr_av.npy'))
rdf_arr_av_bck = np.load(os.path.join(out_path_bck, 'rdf_arr_av.npy'))
rvals = np.load(os.path.join(out_path_fwd, 'rvals.npy'))
act_bck = act_bck #[act_idx_bck_include_list]
act_fwd = act_fwd #[act_idx_fwd_include_list]
rdf_arr_av_fwd = rdf_arr_av_fwd #[act_idx_fwd_include_list]
rdf_arr_av_bck = rdf_arr_av_bck #[act_idx_bck_include_list]

# init dataframe with index rvals and name xvals_name

df = pd.DataFrame(index=rvals/LX,)
# name index
df.index.name = xval_name


for j, act in enumerate(act_include_list):
    try:
        act_idx_fwd = np.argwhere(act_fwd == act)[0][0]

        col_name = f'dir_corr_fwd_(T={act})'
        df[col_name] = rdf_arr_av_fwd[act_idx_fwd, 0,:]
    except:
        pass
    try:
        act_idx_bck = np.argwhere(act_bck == act)[0][0]

        col_name = f'dir_corr_bck_(T={act})'
        df[col_name] = rdf_arr_av_bck[act_idx_bck, 0,:]
    except: 
        pass

# set dtype to float 
df = df.astype(float)
print(df.head(15))
df.to_csv(csv_save_path)
print('Saved csv to', csv_save_path)

In [ ]:
lambda_val = 0
normalize=True
ninfo = 1000

i = lambda_val
fig_num = 3
fig_let = 'c' if i==0 else 'd' #string.ascii_lowercase[i]

xval_name = 'time_step'
csv_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', f'{mode}', f'fig{fig_num}{fig_let}.csv')
os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)

xmask_list = []
yvals_list = []

normalizer = LX if normalize else 1
out_path_all = os.path.join(data_dir, f'{folder_name}\\{mode}\\na{LX}')
out_path_all_wet = os.path.join(data_dir, f'diff_lambda_periodic\\wet\\na{LX}') 

out_path_main = os.path.join(out_path_all, f'{lambda_val}')
out_path_fwd = os.path.join(out_path_main, 'fwd')
out_path_bck = os.path.join(out_path_main, 'bck')

act_bck = np.load(os.path.join(out_path_bck, 'act_bck.npy'))
act_fwd = np.load(os.path.join(out_path_fwd, 'act_fwd.npy'))
nn_arr_fwd = np.load(os.path.join(out_path_fwd, 'nn_arr.npy'))
nn_arr_bck = np.load(os.path.join(out_path_bck, 'nn_arr.npy'))
def_arr_bck = np.load(os.path.join(out_path_bck, 'def_arr.npy'))
def_arr_fwd = np.load(os.path.join(out_path_fwd, 'def_arr.npy'))

out_path_main_wet = os.path.join(out_path_all_wet, f'{lambda_val}')
out_path_fwd_wet = os.path.join(out_path_main_wet, 'fwd')
out_path_bck_wet = os.path.join(out_path_main_wet, 'bck')
act_bck_wet = np.load(os.path.join(out_path_bck_wet, 'act_bck.npy'))
act_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'act_fwd.npy'))
nn_arr_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'nn_arr.npy'))
nn_arr_bck_wet = np.load(os.path.join(out_path_bck_wet, 'nn_arr.npy'))
def_arr_bck_wet = np.load(os.path.join(out_path_bck_wet, 'def_arr.npy'))
def_arr_fwd_wet = np.load(os.path.join(out_path_fwd_wet, 'def_arr.npy'))

nacts=len(marker_list)
act_include_list = [0.045,0.055,0.05] if lambda_val == 0 else [0.004,0.005]

for j, temp in enumerate(act_include_list):
    if temp in act_bck_replace_list[i]:
        act_replace = temp
        act_idx_wet = np.argwhere(act_bck_wet == act_replace)[0][0]
        rnn_arr = nn_arr_bck_wet[:,0,act_idx_wet,]
        defect_arr = def_arr_bck_wet[:,act_idx_wet,]
        print("Using wet data for", temp)
    else:
        actidx = np.argwhere(act_bck == temp)[0][0]
        rnn_arr = nn_arr_bck[:,0,actidx,]
        defect_arr = def_arr_bck[:,actidx,]
    mask_bck = rnn_arr > 0
    normalizer = LX / np.sqrt(defect_arr[mask_bck]) if normalize else 1# LX
    xvals = np.arange(0, Nframes * ninfo, ninfo)[mask_bck]
    yvals = rnn_arr[mask_bck] / normalizer
    xmask_list.append(mask_bck)
    yvals_list.append(yvals)


xvals_tot = np.arange(0, Nframes * ninfo, ninfo)
df = pd.DataFrame(index=xvals_tot   )
# name index
df.index.name = xval_name


for j, act in enumerate(act_include_list):
    try:
        act_idx_fwd = np.argwhere(act_fwd == act)[0][0]

        col_name = f'rnn_av_bck_normalized_(T={act})'
        df[col_name] = np.nan
        df.loc[xmask_list[j], col_name] = yvals_list[j]
    except:
        pass


# set dtype to float 
df = df.astype(float)
#print(df.head(15))
df.to_csv(csv_save_path)
print('Saved csv to', csv_save_path)
df.head(25)

In [ ]:
#mode='wetmerge'
normalize=True
ninfo = 1000
L_list = [256]

lambda_val = 1
normalize=True
plot_backwards=True


i = lambda_val
fig_num = 1
fig_let = 'i' # if i==0 else 'f' #string.ascii_lowercase[i]

xval_name = 'effective_temp'
csv_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', f'{mode}', f'fig{fig_num}{fig_let}.csv')
os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)

#####
act_idx_min_fwd=0
act_idx_min_bck = 0
act_idx_min_fwd = act_idx_min_fwd
null_idx_bck = 2

out_path_all = os.path.join(data_dir, f'{folder_name}\\{mode}\\na{LX}')
out_path_main = os.path.join(out_path_all, f'{lambda_val}') 
if mode == 'dryvl':
    out_path_fwd = os.path.join(data_dir, f'dry_periodic_long\\dry\\na{LX}\\{lambda_val}\\fwd')
else:
    out_path_fwd = os.path.join(out_path_main, 'fwd')

#'C:\\Users\\Simon Andersen\\Projects\\Projects\\Thesis\\NematicAnalysis\\data\\passive\\dry_periodic_long\\dry\\na256\\1\\fwd'
out_path_bck = os.path.join(out_path_main, 'bck')

act_bck = np.load(os.path.join(out_path_bck, 'act_bck.npy'))
act_fwd = np.load(os.path.join(out_path_fwd, 'act_fwd.npy'))
conv_list_fwd = np.load(os.path.join(out_path_fwd, 'conv_list.npy'))
conv_list_bck = np.load(os.path.join(out_path_bck, 'conv_list.npy'))
act_bck = act_bck[act_min_idx_bck:]

nn_arr_fwd_av = np.load(os.path.join(out_path_fwd, 'nn_av.npy'))
nn_arr_bck_av = np.load(os.path.join(out_path_bck, 'nn_av.npy'))[act_min_idx_bck:]

normalizer = LX ** 2
uncertainty_multiplier = 20
def_arr_fwd = np.load(os.path.join(out_path_fwd, 'def_arr.npy'))
def_arr_bck = np.load(os.path.join(out_path_bck, 'def_arr.npy'))[act_min_idx_bck:]
def_arr_fwd_av = np.nan * np.zeros((len(act_fwd), 2))
def_arr_bck_av = np.nan * np.zeros((len(act_bck), 2))

for j, T in enumerate(act_fwd):
    arr_res = def_arr_fwd[int(conv_list_fwd[j]):,j]
    nsamples = arr_res.size
    def_arr_fwd_av[j,0] = np.nanmean(arr_res)
    def_arr_fwd_av[j,1] = np.nanstd(arr_res, ddof = 1,) / np.sqrt(nsamples / uncertainty_multiplier)
    
for j, T in enumerate(act_bck):
    arr_res = def_arr_bck[int(conv_list_bck[j]):,j]
    nsamples = arr_res.size
    def_arr_bck_av[j,0] = np.nanmean(arr_res)
    print(T, np.nanstd(arr_res, ddof = 1,))
    def_arr_bck_av[j,1] = np.nanstd(arr_res, ddof = 1,) / np.sqrt(nsamples / uncertainty_multiplier)

#nanmask_fwd = def_arr_fwd_av[:,0] > 0
#nanmask_bck = def_arr_bck_av[:,0] > 0
#def_arr_fwd_av[~nanmask_fwd,:] = 0 #[0,0]# [np.nan, np.nan]
#def_arr_bck_av[~nanmask_bck,:] = 0 #[np.nan, np.nan]
def_arr_fwd_av /= normalizer
def_arr_bck_av /= normalizer

if null_idx_bck > 0: def_arr_bck_av[:null_idx_bck,:] = 0.0*1e0

if mode=='dry' or mode=='dryvl':
    act_idx_min_bck = 0
    act_idx_min_fwd = 0
    if null_idx_bck>0:
        act_fwd = np.concatenate((act_bck[:null_idx_bck], act_fwd))
        def_arr_fwd_av = np.concatenate((def_arr_bck_av[:null_idx_bck,:], def_arr_fwd_av))
if mode=='wetmerge':
    act_idx_min_bck = 0
    act_idx_min_fwd = 0
    if null_idx_bck>0:
        act_fwd = np.concatenate((act_bck[:null_idx_bck], act_fwd))
        def_arr_fwd_av = np.concatenate((def_arr_bck_av[:null_idx_bck,:], def_arr_fwd_av))


temp_list_fwd = act_fwd = act_fwd[act_idx_min_fwd:]
temp_list_bck = act_bck[act_idx_min_bck:]
#temp_list_fwd.append(act_fwd[act_fwd_mask])
print(temp_list_bck, temp_list_fwd)
defs_fwd = def_arr_fwd_av[act_idx_min_fwd:,:]
defs_bck = def_arr_bck_av[act_idx_min_bck:,:]

temp_list_tot = np.unique(np.concatenate((temp_list_fwd, temp_list_bck)))
xval_name= 'effective_temp'

df = pd.DataFrame(index=temp_list_tot)
# name index
df.index.name = xval_name

for j, (direc, temp_list_dir, yval_list_dir) in enumerate(zip(['fwd', 'bck'], [temp_list_fwd, temp_list_bck], [defs_fwd, defs_bck ])):
    if direc == 'bck' and not plot_backwards:
        continue
    # extend temp_list_dir to total temp list
    temp_mask_ext = False * np.ones_like(temp_list_tot, dtype=bool)
    # if temp in temp_list_fwd[j], set mask to True
    for k, temp in enumerate(temp_list_tot):
        if temp in temp_list_dir:
            temp_mask_ext[k] = True
    col_name_av = f'densisty_av_{direc}'
    col_name_std = f'density_sem_{direc}'

    df[col_name_av] = np.nan
    df[col_name_std] = np.nan
    df.loc[temp_mask_ext, col_name_av] = yval_list_dir[:,0]
    df.loc[temp_mask_ext, col_name_std] = yval_list_dir[:,1]


# set dtype to float 
df = df.astype(float)
df.to_csv(csv_save_path)
print('Saved csv to', csv_save_path)
df.head(15)

#### SAVE FRAMES / LATTICE DATA AS NPZ


In [ ]:
if mode == 'dry':
        LXB, LYB = 12, 75
        Lstart = LXB
        Lend = LXB + LX/1.5 - 20
elif mode == 'wet' and lambda_val == 1:
    LXB, LYB = 75, 0
    Lstart = LXB
    Lend = LXB + LX/1.5 - 22
elif mode in ['wet', 'wetmerge'] and lambda_val == 0:
    LXB, LYB = 10, 40
    Lstart = LXB
    Lend = LXB + LX/1.5 - 0

for j, frame in enumerate(frame_list):

    fig_num = 1
    fig_let = string.ascii_lowercase[j+9]

    npz_save_path = os.path.join(research_dir, 'DefectUnbinding', 'Data', f'{mode}', f'fig{fig_num}{fig_let}.npz')
    os.makedirs(os.path.dirname(npz_save_path), exist_ok=True)

    Qxx = frame.QQxx.reshape(frame.LX, frame.LY)
    Qyx = frame.QQyx.reshape(frame.LX, frame.LY)
    _, director_x, director_y = mp.nematic.nematicPy.get_director(Qxx, Qyx)

    defect_dict = mp.nematic.nematicPy.get_defects(Qxx, Qyx, frame.LX, frame.LY)
    defect_array = get_defect_arr_from_frame(defect_dict, return_charge=True)
    if defect_array is None:
        defect_array = np.empty((0,3))

    Lx_plotting_bounds = [Lstart, Lend]
    Ly_plotting_bounds = [Lstart + LYB, Lend + LYB]

    print(Lx_plotting_bounds, Ly_plotting_bounds, defect_array.shape)

    np.savez_compressed(npz_save_path,
                        director_x=director_x, director_y=director_y, defect_array=defect_array,
                        Lx_plotting_bounds=Lx_plotting_bounds, Ly_plotting_bounds=Ly_plotting_bounds)
    print(f'Saved data to {npz_save_path}')